# Fase 7 EDR: Signal Feature extraction
 
Al fin y al cabo lo que estamos buscando es eficiencia en los aterrzajes. Podemos analizar partes de la `serie temporal` que es cada experimento, y sacar variables significativas, como cuanta desviación hay en el eje x, velocidad en el momento del aterrizaje, etc.
 
Atributos a implementar:

1. <span style="color:green">No es necesario:</span>No sean extremadamente largos
2. (Viejo) Si acaso consiguen terminar aterrizar y si rebotan
3. (terminar): Tengan poca variabilidad en el eje x (calcular la variación de X sobre la posición inicial que es cero)
4. Si queremos tener un un aterrizaje eficiente, tenemos que ver cual es el uso del booster, porque cuanto menos se use mejor:
    * Uso medio del booster
    * Varianza del booster
5. (Hecho)Varianza y media de la velocidad angular
6. (Hecho)La posición de Y debe terminar en cero (valor medio de los últimos instantes de Y)
7. (Hecho)La velocidad angular debe terminar en cero (valor medio de los últimos instantes de la velocidad angular)
8. Analizar la posición de la nave respecto a x antes y después del aterrizaje. Cualquier variación tras haber tocado el suelo probablemente implica baja eficiencia. La idea es mantenerse en 0 o alcanzar esa posición antes de tocar el suelo.
9. En el caso de posiciones de x con comportaminetos extraños, quizá podemos reconstruir los cambios de posición usando la velocidad y un ODE. (La velocidad es la derivada de la función de la posición) [<span style="color:cyan">Apunte:</span> Sólo nos importa la variación de X, lo que podemos hacer integrando]
10. Utilizar los outputs de `getLandedExperiments.py` para crear una variable que indique si el experimento ha aterrizado correctamente o no.
11. <span style="color:red">Dudoso:</span> Suavidad en las funciones de aceleración

## Cosas que estudiar

* Ver cómo acaban los experimentos que no aterrizan
* Mejorar la smooth function, cuando la Y no cae drásticamente a 0 pero sí a un numero cercano, no se corrige el valor.

### Implementaciones
#### 1. No sean extremadamente largos
Para poder evaluar esto, tenemos que ver los boxplots the cada uno de los experimentos, y sería buena idea que los separasemos por si son capaces de aterrizar o no.
Vamos a sacar si han aterrizado o no usando `fileAggregator.py` y la función `expLanded` de `f7utils.py`.

In [ ]:
library(dplyr)
library(ggplot2)

In [30]:
#Remove the duplicate index variable
summarylanded <- read.csv(file = "data/CheckLandingV3.csv") %>% select(-X)
summarylanded$lands <- as.factor(summarylanded$lands)
head(summarylanded)

,filename,total_timesteps,gravity,wind_power,turbulence_power,efficiency,lands,landingPerc,nbounces,angVelMean,angVelVar,finalYmean,finalYvelMean,finalAng_velMean,mBoosterMean,mBoosterVar
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,experiment_1.csv,185,"-5,375066","14,337393","1,9027891","415,69",1,"0,01408599",0,"0,0010759721","0,03220839","-7,820368e-05","3,676812e-10","-5,961901e-05","-0,2466446","0,2565455"
2,experiment_2.csv,428,"-3,958084","16,376438","1,6979323","104,37",1,"0,02112898",3,"0,0963510133","0,09787787","2,258354e-02","-1,086688e-01","7,389720e-02","-0,2655977","0,2757078"
3,experiment_4.csv,193,"-3,640152","1,614948","0,1922136","411,14",1,"0,01450646",0,"-0,0002423267","0,00542839","-9,307528e-04","5,258663e-11","-6,075876e-04","-0,3118868","0,2335856"
4,experiment_5.csv,166,"0,000000","0,000000","0,2421779","414,93",1,"0,01377063",0,"-0,0008201633","0,03433285","-5,592145e-04","-1,160412e-09","2,446833e-03","-0,1429032","0,3100201"
5,experiment_6.csv,172,"-5,289123","15,771525","1,4538064","373,85",1,"0,01471670",0,"-0,0006702543","0,01419187","-5,106763e-04","-1,961436e-10","4,466594e-03","-0,2123328","0,2653827"
6,experiment_7.csv,166,"-7,079790","16,413726","0,9369702","415,83",1,"0,01419111",1,"-0,0004402284","0,05577933","-7,717527e-04","2,220661e-10","4,154155e-03","-0,1135715","0,2999449"


In [ ]:
# how many in the training set didn't land?
length(which(summarylanded$lands == 0))
nrow(summarylanded)

In [ ]:
summarylandedbp <- summarylanded
levels(summarylandedbp$lands) <- c("Not Landed", "Landed")

ggplot(summarylandedbp) + aes(y = total_timesteps) +
    geom_boxplot() + facet_wrap(.~lands) + theme_minimal() +
    theme(legend.position = "bottom") +
    labs(title = "Length of the experiments", y = "timesteps")

ggplot(summarylandedbp) + aes(x = total_timesteps) + geom_histogram() + 
    facet_wrap(.~lands) + theme_minimal() +
    theme(legend.position = "bottom") +
    labs(title = "Length of the experiments", y = "timesteps")

Como tenemos un problema con los outliers de los casos en los que la nave aterriza en el training set, vamos a hacer primero una comparativa con el testing set para ver si es significante.

In [19]:
#Remove the duplicate index variable
summarylandedtesting <- read.csv(file = "data/CheckLandingV2test.csv") %>%
                        select(-X)

summarylandedtesting$lands <- as.factor(summarylandedtesting$lands)

head(summarylandedtesting)

,filename,total_timesteps,gravity,wind_power,turbulence_power,lands
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<fct>
1,experiment_0.csv,156,"-10,363594","9,555678","0,4416887",1
2,experiment_3.csv,252,"-2,599651","7,094931","0,7720043",1
3,experiment_8.csv,146,"-10,768399","5,744031","0,8987043",1
4,experiment_10.csv,178,"-7,815608","19,640269","1,6313483",1
5,experiment_14.csv,193,"-3,833533","11,007721","1,5779704",1
6,experiment_19.csv,204,"-3,464858","0,000000","1,1071370",1


In [ ]:
# how many in the testing set didn't land?
length(which(summarylandedtesting$lands == 0))
nrow(summarylandedtesting)

In [ ]:
summarylandedbptesting <- summarylandedtesting
levels(summarylandedbptesting$lands) <- c("Not Landed", "Landed")

ggplot(summarylandedbptesting) + aes(y = total_timesteps) +
    geom_boxplot() + facet_wrap(.~lands) + theme_minimal() +
    theme(legend.position = "bottom") +
    labs(title = "Length of the experiments in the testing set", y = "timesteps")

Como podemos ver, la distribucion de ambos train and test sets para los timesteps, es muy parecida, por no decir idéntica, si la diferencia de experimentos entre uno y otro fuera menor. Los boxplots reflejan claramente como tanto los outliers como los no outliers son exactamente iguales en distribución en la comparacion entre training y testing, por lo que debemos tener mucho cuidado a la hora de borrar outliers.

#### Efficiency distributed by landings

In [ ]:
ggplot(summarylandedbp) + aes(x = efficiency) + geom_histogram(bins = 120) + 
    facet_wrap(.~lands) + theme_minimal() +
    theme(legend.position = "bottom") +
    labs(title = "Efficiency of the experiments", y = "count")

#### Experimento 6
Podemos notar algo extraño en el experimento 6, en el sentido de que tiene una buena marca de eficiencia, pero al parecer no aterriza. 

Vamos a utilizar la función de python `PlotExperiment` para analizar el experimento 6. Para poder usar código de python dentro de R necesitaremos la librería `reticiulate`.

In [ ]:
if (!require("reticulate")){
    install.packages("reticulate")
    library(reticulate)
} else {
    library(reticulate)
}

In [ ]:
py_run_string('import pandas as pd')
py_run_string('from f7utils import smoothY_pos,plotExperiment')
py_run_string('exp6 = smoothY_pos(pd.read_csv("data/train/experiment_6.csv"))')
py_run_string("plotExperiment(exp6)")

Como podemos ver, deberíamos también analizar si un experimento ha aterrizado por el hecho de si se quedan ambas `y_vel` y `ang_vel` estables en 0

In [21]:
exp6 = read.csv(file = "data/train/experiment_6.csv")
tail(exp6)

,x_pos,y_pos,x_vel,y_vel,angle,ang_vel,leg_1,leg_2,main_booster,lat_booster
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
168,"-0,07802667","-0,0005106831","6,959350e-08","-7,587821e-11","-0,005813542","-8,400928e-08",0,1,"-0,9228973","-0,057367980"
169,"-0,07802667","-0,0005106831","1,582299e-08","-1,231156e-09","-0,005813544","-5,847227e-08",0,1,"-0,9041787","-0,031219900"
170,"-0,07802667","-0,0005106831","-3,847062e-08","-2,008729e-09","-0,005813538","9,187677e-08",0,1,"-0,8049090","-0,041426897"
171,"-0,07802667","-0,0005106831","-4,967608e-08","3,368851e-10","-0,005813533","9,741294e-08",0,1,"-0,9020065","-0,018550575"
172,"-0,07802667","-0,0005106831","-4,119448e-08","3,269379e-10","-0,005813526","9,316055e-08",0,1,"-0,9297094","-0,001113772"
173,"-0,07802667","-0,0005106831","0,000000e+00","0,000000e+00","-0,005813521","0,000000e+00",0,1,"-0,9297094","-0,001113772"


## Linear regressions
To test the significance of the variables:

In [34]:
summarylandedlm <- summarylanded %>% select(-filename)

mod1 <- lm(data = summarylandedlm, efficiency ~ (.)^2)

# modGood <- lm(data = summarylandedlm,
#               efficiency ~ total_timesteps + 
#               wind_power + lands +
#               total_timesteps * gravity + total_timesteps * wind_power +
#               total_timesteps * turbulence_power + total_timesteps * lands +
#               gravity * lands + wind_power * wind_power*lands + turbulence_power*lands
# )

In [33]:
summary(mod1)

Warning message in printCoefmat(coefs, digits = digits, signif.stars = signif.stars, :
“NAs introduced by coercion”



Call:
lm(formula = efficiency ~ (. - finalAng_velMean)^2, data = summarylandedlm)

Residuals:
    Min      1Q  Median      3Q     Max 
-651,14   -2,24    2,56    7,13  686,17 

Coefficients: (2 not defined because of singularities)
                                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)                       3,444e+02  9,379e+02   0,367  0,71349    
total_timesteps                  -9,811e-01  3,641e-01  -2,694  0,00706 ** 
gravity                          -7,248e+01  5,161e+01  -1,404  0,16020    
wind_power                        1,006e+02  1,891e+01   5,324 1,04e-07 ***
turbulence_power                  9,202e+02  1,896e+02   4,854 1,23e-06 ***
lands1                            1,205e+02  9,352e+02   0,129  0,89751    
landingPerc                      -2,445e+01  9,488e+02  -0,026  0,97945    
nbounces                          9,816e+00  4,963e+00   1,978  0,04797 *  
angVelMean                       -7,030e+03  2,491e+03  -2,821  0,00479 ** 
angVelV

```python
def stdXdeviation(exp: pd.DataFrame) -> float:
    var = sum(exp.x_pos**2)/(exp.shape[0]-1)
    return np.sqrt(var)
```